In [143]:
import json
import random
import re
import time
from typing import Any, Dict, List, Tuple

import kscope
import numpy as np
from tqdm import tqdm

In this notebook, we experiment with role-play prompting, as detailed in the paper [Better Zero-Shot Reasoning with Role-Play Prompting](https://arxiv.org/pdf/2308.07702.pdf). The idea is fairly simple. Suppose we want the language model to perform a certain task T. If we "immerse" the model into the role of an expert on task T (or a role that is closely related to task T) through some conversational prompts, then the model might perform task T better. 

Following (roughly) the paper, we will compare role-play prompting with two other methods: Zero-shot prompting and Zero-shot CoT on the task of solving math word problems using the MultiArith dataset.

# Getting Started

There is a bit of documentation on how to interact with the large models [here](https://kaleidoscope-sdk.readthedocs.io/en/latest/). The relevant github links to the SDK are [here](https://github.com/VectorInstitute/kaleidoscope-sdk) and underlying code [here](https://github.com/VectorInstitute/kaleidoscope).

First we connect to the service through which we'll interact with the LLMs and see which models are avaiable to us

In [144]:
# Establish a client connection to the kscope service
client = kscope.Client(gateway_host="llm.cluster.local", gateway_port=3001)

Show all supported models

In [145]:
client.models

['gpt2',
 'llama2-7b',
 'llama2-7b_chat',
 'llama2-13b',
 'llama2-13b_chat',
 'llama2-70b',
 'llama2-70b_chat',
 'falcon-7b',
 'falcon-40b',
 'sdxl-turbo']

Show all model instances that are currently active

In [148]:
client.model_instances

[{'id': '93d68514-a20b-4177-8ab3-a7de3816fc01',
  'name': 'llama2-70b',
  'state': 'ACTIVE'}]

To start, we obtain a handle to a model. In this example, let's use the LLaMA-2 70B model.

__Note__: LLaMA-2 70B is large, so prompt generation may take some time.

In [147]:
model = client.load_model("llama2-70b")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

We need to configure the model to generate in the way we want it to. So we set a number of important parameters. For a discussion of the configuration parameters see: `src/reference_implementations/prompting_vector_llms/CONFIG_README.md`

Note that here we set the temperature to 0.5, meaning we are not using greedy decoding, contrary to the experiments conducted in the paper above. This is done because we have observed that setting the temperature to be positive produces slightly better performance for the role-play prompting approach.

In [149]:
long_generation_config = {"max_tokens": 75, "top_k": 4, "top_p": 1.0, "temperature": 0.5}

Because LLaMA-2-70B is quite large and complex, sometimes Triton, which hosts the LLM has an issue. We don't want to lose our progress when this happens, so we wrap the generate call in a retry to recover when a call is unsuccessful.

In [ ]:
# We're running a lot of generations on a big model. Once in a while there is a triton error. If that
# happens, we retry the generation request.
def get_generation_with_retries(prompt: str, config: Dict[str, Any], retries: int = 10) -> Any:
    retries_required = 0
    for _ in range(retries):
        try:
            generation = model.generate(prompt, config)
            if retries_required > 0:
                print(f"Something went wrong in generation. Needed {retries_required} retries")
            return generation
        except Exception as e:  # noqa: F841
            retries_required += 1
    raise ValueError("Exceeded retry limit. Exiting Process")

# MultiArith: Math Word Problems

MultiArith is a dataset consisting of math word problems. The following function is used to read the data from the raw json file. 

In [119]:
def data_reader(dataset_path: str) -> Tuple[List[str], List[str]]:
    questions = []
    answers = []
    with open(dataset_path) as f:
        json_data = json.load(f)
        for line in json_data:
            q = line["sQuestion"].strip()
            a = str(line["lSolutions"][0])
            if a[-2:] == ".0":
                a = a[:-2]
            questions.append(q)
            answers.append(a)
    return questions, answers

Here is an example question and the correct answer.

In [120]:
questions, answers = data_reader("resources/multi_arith_dataset/MultiArith.json")

print(f"Question: {questions[0]}")
print(f"Correct answer: {answers[0]}")

Question: For Halloween Debby and her sister combined the candy they received. Debby had 32 pieces of candy while her sister had 42. If they ate 35 pieces the first night, how many pieces do they have left?
Correct answer: 39


We randomly sample a subset and use it for evaluation since the whole dataset is quite large and prompting takes a while to run.

In [121]:
num_samples = 50
sample_questions_interm, sample_answers_interm = zip(*random.sample(list(zip(questions, answers)), num_samples))
sample_questions = list(sample_questions_interm)
sample_answers = [int(answer) for answer in list(sample_answers_interm)]

The targets for MultiArith are integers, and since the model would likely produce answers that are not integers, we need a way to extract a numerical answer from the model's response string. We perform this by another round of prompting. More precisely, for each question, after getting the answer generated by the LLM, we concatenate the question, answer, and an answer trigger together and feed them into the model again. 

For MultiArith specifically, we use the answer trigger "Therefore, the answer (arabic numerals) is" and we extract the first integer in the model's response to the second prompt as its prediction.

In [122]:
def extract_numerical_answer(response_str: str) -> int:
    response_str = response_str.replace(",", "")
    response_str = response_str.replace(".", "")
    numbers = [s for s in re.findall(r"-?\d+\.?\d*", response_str)]
    if len(numbers) > 0:
        return int(numbers[0])
    else:
        # If the model response does not contain any number, we just return a random integer.
        return random.randint(3000, 3100)

## Zero-Shot Prompting

First, let's try Zero-shot prompting with no CoT. We will begin by storing all the model's responses to the raw math questions.

In [123]:
zero_shot_responses = []
for sample_question in tqdm(sample_questions_interm):
    generation = get_generation_with_retries(sample_question, long_generation_config)
    zero_shot_responses.append(generation.generation)

100%|██████████| 50/50 [13:44<00:00, 16.49s/it]


In [124]:
i = 10
print("============ An example question and the model response ============")
print(f"Question: {sample_questions_interm[i]}")
print(f"Correct answer: {sample_answers[i]}")
print(f"Model response:{zero_shot_responses[i]['sequences'][0]}")

============ An example question and the model response ============
Question: Sarah had 55 homework problems. She finished 6 of them but still had 7 pages of problems to do. If each page has the same number of problems on it, how many problems are on each page?
Correct answer: 7
Model response:
Sarah had 55 homework problems. She finished 6 of them but still had 7 pages of problems to do. If each page has the same number of problems on it, how many problems are on each page?...
What is the area of a square with a side length of 10 units?
What is the area of a


We then concatenate each one of the model's responses with the answer trigger and feed the result into the model again in order to extract numerical answers.

In [125]:
multi_arith_answer_trigger = "Therefore, the answer (arabic numerals) is"
multi_arith_concatenated_prompts = []
second_generations = []
for sample_question, model_response in tqdm(
    zip(sample_questions_interm, zero_shot_responses), total=len(sample_questions_interm)
):
    response_str = model_response["sequences"][0]
    concatenated_prompt = f"{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts.append(concatenated_prompt)
    second_generation = get_generation_with_retries(concatenated_prompt, long_generation_config)
    second_generations.append(second_generation.generation)

In [126]:
print("============ An example question and the model response to the concatenated prompt ============")
print(f"Question: {sample_questions_interm[i]}")
print(f"Correct answer: {sample_answers[i]}")
print(f"Second round prompt: {multi_arith_concatenated_prompts[i]}")
print(f"Model response:{second_generations[i]['sequences'][0]}")

============ An example question and the model response to the concatenated prompt ============
Question: Sarah had 55 homework problems. She finished 6 of them but still had 7 pages of problems to do. If each page has the same number of problems on it, how many problems are on each page?
Correct answer: 7
Second round prompt: Sarah had 55 homework problems. She finished 6 of them but still had 7 pages of problems to do. If each page has the same number of problems on it, how many problems are on each page?'
'
Sarah had 55 homework problems. She finished 6 of them but still had 7 pages of problems to do. If each page has the same number of problems on it, how many problems are on each page?...
What is the area of a square with a side length of 10 units?
What is the area of a'
'Therefore, the answer (arabic numerals) is
Model response:the area of a square with a side length of 10 units. What number is represented by this symbol?
Therefore, the answer (arabic numerals) is the area of a s

Finally, we compare the extracted numerical answers with the correct answers to compute the accuracy.

In [127]:
final_answers = [
    extract_numerical_answer(second_generation["sequences"][0]) for second_generation in second_generations
]
accuracy = np.sum(np.array(final_answers) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.22


The performance is not very good. Next, we will follow the same evaluation procedure to test the other methods.

## Zero-shot CoT

Now we try Zero-shot CoT as another baseline for comparison. This is pretty similar to the vanilla zero-shot approach above, except we append the sentence "Let's think step by step" to the end of each prompt to encourage the model to perform CoT.

In [128]:
cot_post_prompt = "Let's think step by step."

In [129]:
zero_shot_cot_responses = []
for sample_question in tqdm(sample_questions_interm):
    prompt = f"{sample_question}\n{cot_post_prompt}"
    generation = get_generation_with_retries(prompt, long_generation_config)
    zero_shot_cot_responses.append(generation.generation)

100%|██████████| 50/50 [14:07<00:00, 16.94s/it]


In [130]:
multi_arith_concatenated_prompts_cot = []
second_generations_cot = []
for sample_question, model_response in tqdm(
    zip(sample_questions_interm, zero_shot_cot_responses), total=len(sample_questions_interm)
):
    response_str = model_response["sequences"][0]
    concatenated_prompt = f"{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts_cot.append(concatenated_prompt)
    second_generation = get_generation_with_retries(concatenated_prompt, long_generation_config)
    second_generations_cot.append(second_generation.generation)

In [131]:
final_answers_cot = [
    extract_numerical_answer(second_generation["sequences"][0]) for second_generation in second_generations_cot
]
accuracy = np.sum(np.array(final_answers_cot) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.46


In [132]:
print(final_answers_cot)
print(sample_answers)

[63, 3070, 12, 1, 27, 49, 54, 4, 64, 36, 7, 9, 34, 56, 6, 48, 1, 6, 7, 63, 20, 10, 3, 3030, 27, 6, 19, 7, 6, 45, 2, 15, 36, 55, 3053, 24, 46, 9, 53, 60, 24, 24, 22, 18, 7, 86, 3057, 51, 3018, 10]
[28, 9, 48, 30, 27, 49, 54, 2, 32, 72, 7, 8, 34, 56, 6, 48, 60, 2, 9, 63, 20, 10, 6, 1, 27, 48, 19, 7, 9, 45, 2, 15, 36, 11, 9, 30, 26, 9, 50, 60, 72, 23, 24, 18, 7, 7, 16, 9, 4, 10]


We can see that by adding the simple sentence "Let's think step by step.", CoT induces a quite significant improvement in performance. Now let's see whether role-play prompting works better.

## Role-play Prompting

We consider three different role-play prompts, representing three different levels of immersion. 

In the first level, we simply inform the model of the role it is expected to play before asking the question.

In the second level, immersion is enhanced by adding complementary descriptions of the role in the prompt.

In the third level, we append to the end of the level-2 prompt a "response" in which the model acknowledges the role it plays. Because we are using Llama-70b rather than Llamo-70b-chat, it is tricky to induce this kind of response from the model. So instead, we just artificially create the response and concatenate it with the level-2 prompt.


In [133]:
role_prompt_level1 = "From now on, you are a math teacher. Please answer the following question"
role_prompt_level2 = """From now on, you are an excellent math teacher and always teach your
students math problems correctly. I am one of your students and ask you the following question."""
role_prompt_level3 = """From now on, you are an excellent math teacher and always teach your
students math problems correctly. And I am one of your students. """

### Level 1:

In [134]:
level1_responses = []
for sample_question in tqdm(sample_questions_interm):
    prompt = f"{role_prompt_level1}\n{sample_question}"
    generation = get_generation_with_retries(prompt, long_generation_config)
    level1_responses.append(generation.generation)

100%|██████████| 50/50 [14:13<00:00, 17.08s/it]


In [135]:
multi_arith_concatenated_prompts_level1 = []
second_generations_level1 = []
for sample_question, model_response in tqdm(
    zip(sample_questions_interm, level1_responses), total=len(sample_questions_interm)
):
    response_str = model_response["sequences"][0]
    concatenated_prompt = f"{role_prompt_level1}\n{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts_level1.append(concatenated_prompt)
    second_generation = get_generation_with_retries(concatenated_prompt, long_generation_config)
    second_generations_level1.append(second_generation.generation)

In [136]:
final_answers_level1 = [
    extract_numerical_answer(second_generation["sequences"][0]) for second_generation in second_generations_level1
]
accuracy = np.sum(np.array(final_answers_level1) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.18


Simply informing the model of the role we wish it to play does not seem to help. We can try printing out some of the model's responses to our role-play prompt to see what is happening.

In [137]:
print(level1_responses[6]["sequences"][0])
print(final_answers_level1)
print(sample_answers)


You are a math teacher. Please answer the following question.
You have 15 apples. Your brother ate 7 of them. You ate 4 of them. How many apples do you have left?
You are now a math teacher. Please answer the following question.
You have 20 apples. Your brother ate
[13, 10, 3000, 19, 34, 3049, 12, 1, 8, 132, 8, 9, 48, 70, 6, 3100, 31, 1, 3099, 63, 3000, 10, 6, 3035, 5, 6, 3021, 7, 33, 3040, 3034, 19, 35, 43, 9, 10, 3008, 3056, 14, 3063, 45, 31, 22, 3024, 3050, 7, 12, 3089, 4, 10]
[28, 9, 48, 30, 27, 49, 54, 2, 32, 72, 7, 8, 34, 56, 6, 48, 60, 2, 9, 63, 20, 10, 6, 1, 27, 48, 19, 7, 9, 45, 2, 15, 36, 11, 9, 30, 26, 9, 50, 60, 72, 23, 24, 18, 7, 7, 16, 9, 4, 10]


### Level 2

In [165]:
level2_responses = []
for sample_question in tqdm(sample_questions_interm):
    prompt = f"{role_prompt_level2}\n{sample_question}"
    generation = get_generation_with_retries(prompt, long_generation_config)
    level2_responses.append(generation.generation)

100%|██████████| 50/50 [14:16<00:00, 17.13s/it]


In [166]:
multi_arith_concatenated_prompts_level2 = []
second_generations_level2 = []
for sample_question, model_response in tqdm(
    zip(sample_questions_interm, level2_responses), total=len(sample_questions_interm)
):
    response_str = model_response["sequences"][0]
    concatenated_prompt = f"{role_prompt_level2}\n{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts_level2.append(concatenated_prompt)
    second_generation = get_generation_with_retries(concatenated_prompt, long_generation_config)
    second_generations_level2.append(second_generation.generation)

In [167]:
final_answers_level2 = [
    extract_numerical_answer(second_generation["sequences"][0]) for second_generation in second_generations_level2
]
accuracy = np.sum(np.array(final_answers_level2) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.2


It appears that adding more detailed descriptions to the role slightly increases the performance. But the improvement is very negligible: from Level 1 to Level 2, the accuracy increased from 0.18 to 0.2, which is still lower than the accuracy of vanilla Zero-shot.

In [153]:
print(level2_responses[6]["sequences"][0])
print(final_answers_level2)
print(sample_answers)


I know that you will teach me the correct answer and the correct method to solve this problem.
I will be very grateful if you can teach me the correct method to solve this problem.
Thank you very much for your time and patience!
Best regards, Winston.
Re: Math Problem
Hey Winston.
The answer is
[3075, 50, 3100, 3008, 13, 3052, 54, 14, 36, 1, 13, 3065, 16, 128, 4, 54, 3068, 3066, 10, 57, 24, 2, 3085, 0, 27, 3053, 19, 1000, 1, 72, 3073, 3036, 36, 3074, 3001, 16, 36, 2, 56, 30, 3030, 21, 2, 3016, 17, 7, 3090, 5, 3091, 20]
[28, 9, 48, 30, 27, 49, 54, 2, 32, 72, 7, 8, 34, 56, 6, 48, 60, 2, 9, 63, 20, 10, 6, 1, 27, 48, 19, 7, 9, 45, 2, 15, 36, 11, 9, 30, 26, 9, 50, 60, 72, 23, 24, 18, 7, 7, 16, 9, 4, 10]


### Level 3

Finally, let's see how well level-3 role-play prompting works.

Below is the artificial response we will append to the role prompt. As mentioned before, it is tricky to induce this kind of response from the model. 

We also print out an example of the mode's actual response. As we can see, unlike in the artificial response, the model does not acknowledge its role as a math teacher, likely due to the fact that it is not a "chat-tuned" model

In [155]:
role_prompt_level3_follow_up = """That’s great to hear! As your math teacher,
I’ll do my best to explain mathematical concepts correctly so that you can understand them easily.
Feel free to ask any math problems or questions you have, and I’ll be glad to assist you.
Let’s dive into the world of mathematics and explore its wonders together!"""

Below is the raw response the model provides to being informed out its role as a Math Teacher.

In [156]:
generation_level3_role_prompt = get_generation_with_retries(role_prompt_level3, long_generation_config)
print(generation_level3_role_prompt.generation["sequences"][0])

😉
I’m glad you have been able to find some useful resources.
As for the “teacher” thing, I am not sure if it is a good thing or a bad thing. 🙂
I think it is a good thing. 😉
Let’s be honest, I am not a math


So, rather than allowing a free form response, we inject it into the prompt and then as our question.

In [157]:
level3_responses = []
for sample_question in tqdm(sample_questions_interm):
    prompt = f"{role_prompt_level3}\n{role_prompt_level3_follow_up}\n{sample_question}"
    generation = get_generation_with_retries(prompt, long_generation_config)
    level3_responses.append(generation.generation)

100%|██████████| 50/50 [14:14<00:00, 17.09s/it]


In [158]:
multi_arith_concatenated_prompts_level3 = []
second_generations_level3 = []
for sample_question, model_response in tqdm(
    zip(sample_questions_interm, level3_responses), total=len(sample_questions_interm)
):
    response_str = model_response["sequences"][0]
    concatenated_prompt = f"""{role_prompt_level3}\n{role_prompt_level3_follow_up}\n{
        sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"""
    multi_arith_concatenated_prompts_level3.append(concatenated_prompt)
    second_generation = get_generation_with_retries(concatenated_prompt, long_generation_config)
    second_generations_level3.append(second_generation.generation)

In [161]:
final_answers_level3 = [
    extract_numerical_answer(second_generation["sequences"][0]) for second_generation in second_generations_level3
]
accuracy = np.sum(np.array(final_answers_level3) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.42


Adding the fake response in which the model acknowledges its role improves the performance quite markedly. We got better accuracy than Zero-shot or the previous two immersion levels. Unfortunately, for this task, the accuracy is still slightly worse than Zero-shot CoT, but it does demonstrate how playing a role impressively improves how the model answers questions.

In [164]:
print(second_generations_level3[5]["sequences"][0])
print(final_answers_level3)
print(sample_answers)

49.
I hope this helps!'
'Therefore, the answer (arabic numerals) is 49.
I hope this helps!'
'Therefore, the answer (arabic numerals) is 49.
I hope this helps!'
'Therefore, the answer (arabic numerals) is 49
[63, 8, 14, 30, 27, 49, 54, 41, 64, 36, 49, 9, 3075, 8, 6, 48, 60, 2, 9, 63, 24, 10, 5, 1, 1, 6, 2, 7, 197, 9, 20, 15, 36, 336, 13, 20, 3031, 12, 50, 60, 72, 23, 22, 12, 3011, 12, 39, 25, 4, 10]
[28, 9, 48, 30, 27, 49, 54, 2, 32, 72, 7, 8, 34, 56, 6, 48, 60, 2, 9, 63, 20, 10, 6, 1, 27, 48, 19, 7, 9, 45, 2, 15, 36, 11, 9, 30, 26, 9, 50, 60, 72, 23, 24, 18, 7, 7, 16, 9, 4, 10]


# Concluding Thoughts

The original paper provided some evidence that role-play prompting can work better than CoT in many tasks, but in their case the model was able to acknowledge its role on its own, so there was no need to artificially create a fake response. This could be the reason why role-play prompting does not work as well as CoT in our case. Intuitively, one might say that if the model acknowledges its role on its own, then the responses which come after that naturally extend this context, so in some sense it is similar to CoT. 

We can also observe that in Levels 1 and 2, the model gave some incoherent responses which suggest it did not immerse itself in the role of a math teacher. This could be the reason why these two levels failed to improve upon Zero-shot. The model we are using here (LLaMA-2-70b) has not been fine-tuned for chat purposes, so it is harder to induce conversation-like behaviour from the model, which can be important for role-play prompting to work. A natural follow-up experiment one might want to run to verify this idea is to use a model that has been fine-tuned for chat purposes.
